In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
from sklearn.metrics.pairwise import haversine_distances
from math import radians, sin, cos, sqrt, atan2
from geopy.distance import geodesic
import fiona
from shapely.geometry import Point
from geopy.geocoders import Nominatim

In [2]:
dtypes={'data_year': str,	'offense_id':str, 'incident_id':str,'offense_code': str,                                                                            'cargo_theft_flag': str, 'submission_date':str, 'incident_date':str,
        'attempt_complete_flag':str, 'location_id':str, 'cleared_except_id':str,'num_premises_entered':str,
        'method_entry_code':str}

al_crime = pd.read_csv('../data/crime/AL-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
ar_crime = pd.read_csv('../data/crime/AR-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
de_crime = pd.read_csv('../data/crime/DE-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
dc_crime = pd.read_csv('../data/crime/DC-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
fl_crime = pd.read_csv('../data/crime/FL-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
ga_crime = pd.read_csv('../data/crime/GA-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
ky_crime = pd.read_csv('../data/crime/KY-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
la_crime = pd.read_csv('../data/crime/LA-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
md_crime = pd.read_csv('../data/crime/MD-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
mo_crime = pd.read_csv('../data/crime/MO-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
ms_crime = pd.read_csv('../data/crime/MS-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
nc_crime = pd.read_csv('../data/crime/NC-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
oh_crime = pd.read_csv('../data/crime/OH-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
ok_crime = pd.read_csv('../data/crime/OK-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
pa_crime = pd.read_csv('../data/crime/PA-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
sc_crime = pd.read_csv('../data/crime/SC-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
tn_crime = pd.read_csv('../data/crime/TN-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
tx_crime = pd.read_csv('../data/crime/TX-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
va_crime = pd.read_csv('../data/crime/VA-2022/NIBRS_OFFENSE.csv',dtype=dtypes)
wv_crime = pd.read_csv('../data/crime/WV-2022/NIBRS_OFFENSE.csv',dtype=dtypes)


In [12]:

dtypes2={'data_year': str, 'agency_id':str, 'incident_id':str, 'nibrs_month_id':str, 'cargo_theft_flag':str, 'submission_date':str,
         'incident_date':str, 'report_date_flag':str, 'incident_hour':str, 'cleared_except_id':str,	'cleared_except_date':str,
         'incident_status':str,	'data_home':str, 'orig_format':str,	'did':str}
al_incident = pd.read_csv('../data/crime/AL-2022/NIBRS_incident.csv',dtype = dtypes2)
ar_incident = pd.read_csv('../data/crime/AR-2022/NIBRS_incident.csv',dtype = dtypes2)
de_incident = pd.read_csv('../data/crime/DE-2022/NIBRS_incident.csv',dtype = dtypes2)
dc_incident = pd.read_csv('../data/crime/DC-2022/NIBRS_incident.csv',dtype = dtypes2)
fl_incident = pd.read_csv('../data/crime/FL-2022/NIBRS_incident.csv',dtype = dtypes2)
ga_incident = pd.read_csv('../data/crime/GA-2022/NIBRS_incident.csv',dtype = dtypes2)
ky_incident = pd.read_csv('../data/crime/KY-2022/NIBRS_incident.csv',dtype = dtypes2)
la_incident = pd.read_csv('../data/crime/LA-2022/NIBRS_incident.csv',dtype = dtypes2)
md_incident = pd.read_csv('../data/crime/MD-2022/NIBRS_incident.csv',dtype = dtypes2)
mo_incident = pd.read_csv('../data/crime/MO-2022/NIBRS_incident.csv',dtype = dtypes2)
ms_incident = pd.read_csv('../data/crime/MS-2022/NIBRS_incident.csv',dtype = dtypes2)
nc_incident = pd.read_csv('../data/crime/NC-2022/NIBRS_incident.csv',dtype = dtypes2)
oh_incident = pd.read_csv('../data/crime/OH-2022/NIBRS_incident.csv',dtype = dtypes2)
ok_incident = pd.read_csv('../data/crime/OK-2022/NIBRS_incident.csv',dtype = dtypes2)
pa_incident = pd.read_csv('../data/crime/PA-2022/NIBRS_incident.csv',dtype = dtypes2)
sc_incident = pd.read_csv('../data/crime/SC-2022/NIBRS_incident.csv',dtype = dtypes2)
tn_incident = pd.read_csv('../data/crime/TN-2022/NIBRS_incident.csv',dtype = dtypes2)
tx_incident = pd.read_csv('../data/crime/TX-2022/NIBRS_incident.csv',dtype = dtypes2)
va_incident = pd.read_csv('../data/crime/VA-2022/NIBRS_incident.csv',dtype = dtypes2)
wv_incident = pd.read_csv('../data/crime/WV-2022/NIBRS_incident.csv',dtype = dtypes2)

In [13]:
dtypes3={'unitid':str, 'institution_name':str, 'year':int, 'HD2022.Historically Black College or University':str, 'agency_id':str,
         'campus_agency':str, 'city_agency':str, 'county_agency':str}
hbcu_list = pd.read_csv('../data/hbcu_list.csv', dtype=dtypes3)
hbcu_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 8 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   unitid                                           101 non-null    object
 1   institution_name                                 101 non-null    object
 2   year                                             101 non-null    int32 
 3   HD2022.Historically Black College or University  101 non-null    object
 4   agency_id                                        94 non-null     object
 5   campus_agency                                    100 non-null    object
 6   city_agency                                      98 non-null     object
 7   county_agency                                    98 non-null     object
dtypes: int32(1), object(7)
memory usage: 6.0+ KB


In [5]:
hbcu_list['agency_id'].unique()
agency_id_list=['360', '99', '263', '353', '51', '49', '23089', '235', '466',
       '450', '94', '23115', '728', '635', '729', '2268', '2243',
       '2615', '2920', '2539', '3693', '4189', '3995', '3776', '4028',
       '3771', '6675', '7269', '7204', '7295', '7171', '8020', '7926',
       '8029', '8079', '10200', '10207', '10313', '10306', '10174',
       '9621', '11202', '11438', '11077', '11335', '11524', '11204',
       '11118', '11607', '11135', '14382', '14372', '25368', '16403',
       '16203', '17892', '17904', '17860', '17975', '17498', '17944',
       '17871', '18271', '18513', '18747', '18295', '19756', '19856',
       '19869', '19800', '19871', '19413', '19678', '19870', '19284',
       '20431', '20448', '20437', '20453', '20460', '21572', '21494',
       '6807']

In [18]:
al_incident_f=al_incident.loc[al_incident['agency_id'].isin(agency_id_list)]
al_incident_f.reset_index()
ar_incident_f=ar_incident.loc[ar_incident['agency_id'].isin(agency_id_list)]
ar_incident_f.reset_index()
de_incident_f=de_incident.loc[de_incident['agency_id'].isin(agency_id_list)]
de_incident_f.reset_index()
dc_incident_f=dc_incident.loc[dc_incident['agency_id'].isin(agency_id_list)]
dc_incident_f.reset_index()
fl_incident_f=fl_incident.loc[fl_incident['agency_id'].isin(agency_id_list)]
fl_incident_f.reset_index()
ga_incident_f=ga_incident.loc[ga_incident['agency_id'].isin(agency_id_list)]
ga_incident_f.reset_index()
ky_incident_f=ky_incident.loc[ky_incident['agency_id'].isin(agency_id_list)]
ky_incident_f.reset_index()
la_incident_f=la_incident.loc[la_incident['agency_id'].isin(agency_id_list)]
la_incident_f.reset_index()
md_incident_f=md_incident.loc[md_incident['agency_id'].isin(agency_id_list)]
md_incident_f.reset_index()
mo_incident_f=mo_incident.loc[mo_incident['agency_id'].isin(agency_id_list)]
mo_incident_f.reset_index()
ms_incident_f=ms_incident.loc[ms_incident['agency_id'].isin(agency_id_list)]
ms_incident_f.reset_index()
nc_incident_f=nc_incident.loc[nc_incident['agency_id'].isin(agency_id_list)]
nc_incident_f.reset_index()
oh_incident_f=oh_incident.loc[oh_incident['agency_id'].isin(agency_id_list)]
oh_incident_f.reset_index()
ok_incident_f=ok_incident.loc[ok_incident['agency_id'].isin(agency_id_list)]
ok_incident_f.reset_index()
pa_incident_f=pa_incident.loc[pa_incident['agency_id'].isin(agency_id_list)]
pa_incident_f.reset_index()
sc_incident_f=sc_incident.loc[sc_incident['agency_id'].isin(agency_id_list)]
sc_incident_f.reset_index()
tn_incident_f=tn_incident.loc[tn_incident['agency_id'].isin(agency_id_list)]
tn_incident_f.reset_index()
tx_incident_f=tx_incident.loc[tx_incident['agency_id'].isin(agency_id_list)]
tx_incident_f.reset_index()
va_incident_f=va_incident.loc[va_incident['agency_id'].isin(agency_id_list)]
va_incident_f.reset_index()
wv_incident_f=wv_incident.loc[wv_incident['agency_id'].isin(agency_id_list)]
wv_incident_f.reset_index()

,index,data_year,agency_id,incident_id,nibrs_month_id,cargo_theft_flag,submission_date,incident_date,report_date_flag,incident_hour,cleared_except_id,cleared_except_date,incident_status,data_home,orig_format,did
0,29414,2022,21494,150440041,44069296,f,2022-03-11 13:14:31.495,2022-02-11,f,0,6,NaN,ACCEPTED,NaN,F,133188986
1,29415,2022,21494,150440057,44069296,f,2022-03-11 13:14:31.64,2022-02-23,f,0,6,NaN,WARNINGS,NaN,F,133189026
2,29418,2022,21494,159595021,44197993,f,2022-10-11 12:13:31.772,2022-08-22,f,0,6,NaN,ACCEPTED,NaN,F,157013038
3,29434,2022,21494,159595036,44209677,f,2022-10-11 12:13:31.865,2022-09-03,f,0,6,NaN,ACCEPTED,NaN,F,157013062
4,29435,2022,21494,161356818,44197993,f,2022-11-10 18:19:41.91,2022-08-10,f,0,6,NaN,ACCEPTED,NaN,F,161650528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,35214,2022,21572,164059491,44257921,f,2023-01-10 17:32:32.946,2022-12-19,f,16,6,NaN,ACCEPTED,NaN,F,168534209
257,35215,2022,21572,164059208,44257921,f,2023-01-10 17:32:29.265,2022-12-20,f,17,6,NaN,ACCEPTED,NaN,F,168533852
258,35216,2022,21572,164059503,44257921,f,2023-01-10 17:32:33.119,2022-12-20,f,13,6,NaN,ACCEPTED,NaN,F,168534227
259,35217,2022,21572,164059238,44257921,f,2023-01-10 17:32:29.628,2022-12-21,f,9,6,NaN,ACCEPTED,NaN,F,168533894


In [21]:
dfs=[al_incident_f, ar_incident_f, de_incident_f, dc_incident_f, fl_incident_f, 
     ga_incident_f, ky_incident_f, la_incident_f, md_incident_f, mo_incident_f, 
     ms_incident_f, nc_incident_f, oh_incident_f, ok_incident_f, pa_incident_f,
     sc_incident_f, tn_incident_f, tx_incident_f, va_incident_f, wv_incident_f]
crime_dfs = pd.concat(dfs, axis=0, ignore_index=True)
crime_dfs

,data_year,agency_id,incident_id,nibrs_month_id,cargo_theft_flag,submission_date,incident_date,report_date_flag,incident_hour,cleared_except_id,cleared_except_date,incident_status,data_home,orig_format,did
0,2022,49,155372673,44169977,f,2022-07-06 13:41:03.8,2022-01-08,f,19,4,2022-01-08,WARNINGS,NaN,F,145967292
1,2022,49,155377151,44169977,f,2022-07-06 13:41:25.777,2022-01-22,f,21,6,NaN,ACCEPTED,NaN,F,145974034
2,2022,49,166585891,44303376,f,2023-02-14 14:44:55.98,2022-12-26,f,15,6,NaN,WARNINGS,NaN,F,175090033
3,2022,49,166584398,44303376,f,2023-02-14 14:44:38.416,2022-12-16,f,20,6,NaN,WARNINGS,NaN,F,175088154
4,2022,49,161106953,44209975,f,2022-11-04 20:20:25.003,2022-08-26,f,12,6,NaN,WARNINGS,NaN,F,160996787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1298431,2022,21572,164059491,44257921,f,2023-01-10 17:32:32.946,2022-12-19,f,16,6,NaN,ACCEPTED,NaN,F,168534209
1298432,2022,21572,164059208,44257921,f,2023-01-10 17:32:29.265,2022-12-20,f,17,6,NaN,ACCEPTED,NaN,F,168533852
1298433,2022,21572,164059503,44257921,f,2023-01-10 17:32:33.119,2022-12-20,f,13,6,NaN,ACCEPTED,NaN,F,168534227
1298434,2022,21572,164059238,44257921,f,2023-01-10 17:32:29.628,2022-12-21,f,9,6,NaN,ACCEPTED,NaN,F,168533894


In [24]:
grouped_crimes= pd.merge(crime_dfs, hbcu_list, on=['agency_id'], how= 'left')
grouped_crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1597421 entries, 0 to 1597420
Data columns (total 22 columns):
 #   Column                                           Non-Null Count    Dtype 
---  ------                                           --------------    ----- 
 0   data_year                                        1597421 non-null  object
 1   agency_id                                        1597421 non-null  object
 2   incident_id                                      1597421 non-null  object
 3   nibrs_month_id                                   1597421 non-null  object
 4   cargo_theft_flag                                 1597421 non-null  object
 5   submission_date                                  1597421 non-null  object
 6   incident_date                                    1597421 non-null  object
 7   report_date_flag                                 1597421 non-null  object
 8   incident_hour                                    1596567 non-null  object
 9   cleared_excep

In [28]:
grouped_crimes1=grouped_crimes.loc[:,['institution_name','data_year', 'agency_id', 'incident_id',
                'campus_agency', 'city_agency', 'county_agency']]
grouped_crimes1

,institution_name,data_year,agency_id,incident_id,campus_agency,city_agency,county_agency
0,Miles College,2022,49,155372673,n,n,y
1,Miles College,2022,49,155377151,n,n,y
2,Miles College,2022,49,166585891,n,n,y
3,Miles College,2022,49,166584398,n,n,y
4,Miles College,2022,49,161106953,n,n,y
...,...,...,...,...,...,...,...
1597416,Bluefield State University,2022,21572,164059491,n,y,n
1597417,Bluefield State University,2022,21572,164059208,n,y,n
1597418,Bluefield State University,2022,21572,164059503,n,y,n
1597419,Bluefield State University,2022,21572,164059238,n,y,n
